# Beyond Gradient Descent

## The Challenges with Gradient Descent 

The fundamental ideas behind neural networks have existed for decades, but it wasn’t until recently that neural network-based learning models have become mainstream. Our fascination with neural networks has everything to do with their expressiveness, a quality we’ve unlocked by creating networks with many layers. As we have discussed in previous chapters, deep neural networks are able to crack problems that were previously deemed intractable. Training deep neural networks end to end, however, is fraught with difficult challenges that took many technological innovations to unravel, including massive labeled datasets (ImageNet, CIFAR, etc.),  better hardware in the form of GPU acceleration, and several algorithmic discoveries. 

For several years, researchers resorted to layer-wise greedy pre-training in order to grapple with the complex error surfaces presented by deep learning models. These time-intensive strategies would try to find more accurate initializations for the model’s parameters one layer at a time before using mini-batch gradient descent to converge to the optimal parameter settings. More recently, however, breakthroughs in optimization methods have enabled us to directly train models in an end-to-end fashion. 

In this chapter, we will discuss several of these breakthroughs. The next couple of sections will focus primarily on local minima and whether they pose hurdles for successfully training deep models. In subsequent sections, we will further explore the nonconvex error surfaces induced by deep models, why vanilla mini-batch gradient descent falls short, and how modern nonconvex optimizers overcome these pitfalls.

## Local Minima in the Error Surfaces of Deep Networks 

The primary challenge  in optimizing deep learning models is that we are forced to use minimal local information to infer the global structure of the error surface. This is a hard problem because there is usually very little correspondence between local and global structure. Take the following analogy as an example. Let’s assume you’re an ant on the continental United States. You’re dropped randomly on the map, and your goal is to find the lowest point on this surface. How do you do it? If all you can observe is your immediate surroundings, this seems like an intractable problem. If the surface of the US was bowl shaped (or mathematically speaking, convex) and we were smart about our learning rate, we could use the gradient descent algorithm to eventually find the bottom of the bowl. But the surface of the US is extremely complex, that is to say, is a nonconvex surface, which means that even if we find a valley (a local minimum), we have no idea if it’s the lowest valley on the map (the global minimum). In Chapter 2, we talked about how a mini-batch version of gradient descent can help navigate a troublesome error surface when there are spurious regions of magnitude zero gradients. But as we can see in figure, even a stochastic error surface won’t save us from a deep local minimum.

<img src='images/img1.PNG'>

Now comes the critical question. Theoretically, local minima pose a significant issue. But in practice, how common are local minima in the error surfaces of deep networks? And in which scenarios are they actually problematic for training? In the following two sections, we’ll pick apart common misconceptions about local minima.


## Model Identifiability

The first source of local minima is tied to a concept commonly referred to as model identifiability. One observation about deep neural networks is that their error surfaces are guaranteed to have a large—and in some cases, an infinite—number of local minima. There are two major reasons this observation is true. 

The first is that within a layer of a fully-connected feed-forward neural network, any rearrangement of neurons will still give you the same final output at the end of the network. We illustrate this using a simple three-neuron layer in Figure below. As a result, within a layer with n neurons, there are n! ways to rearrange parameters. And for a deep network with l layers, each with n neurons, we have a total of n!l equivalent configurations.

<img src='images/img2.PNG'>

In addition to the symmetries of neuron rearrangements, non-identifiability is present in other forms in certain kinds of neural networks. For example, there is an infinite number of equivalent configurations that for an individual ReLU neuron result in equivalent networks. Because an ReLU uses a piecewise linear function, we are free to multiply all of the incoming weights by any nonzero constant k while scaling all of the outgoing weights by 1/k without changing the behavior of the network. We leave the justification for this statement as an exercise for the active reader.

Ultimately, however, local minima that arise because of the non-identifiability of deep neural networks are not inherently problematic. This is because all nonidentifiable configurations behave in an indistinguishable fashion no matter what input values they are fed. This means they will achieve the same error on the training, validation, and testing datasets. In other words, all of these models will have learned equally from the training data and will have identical behavior during generalization to unseen examples.

Instead, local minima are only problematic when they are spurious. A spurious local minimum corresponds to a configuration of weights in a neural network that incurs a higher error than the configuration at the global minimum. If these kinds of local minima are common, we quickly run into significant problems while using gradientbased optimization methods because we can only take into account local structure. 

## How Pesky Are Spurious Local Minima in Deep Networks?

For many years, deep learning practitioners blamed all of their troubles in training deep networks on spurious local minima, albeit with little evidence. Today, it remains an open question whether spurious local minima with a high error rate relative to the global minimum are common in practical deep networks. However, many recent studies seem to indicate that most local minima have error rates and generalization characteristics that are very similar to global minima. 

One way we might try to naively tackle this problem is by plotting the value of the error function over time as we train a deep neural network. This strategy, however, doesn’t give us enough information about the error surface because it is difficult to tell whether the error surface is “bumpy,” or whether we merely have a difficult time figuring out which direction we should be moving in. 

To more effectively analyze this problem, Goodfellow et al. (a team of researchers collaborating between Google and Stanford) published a paper in 2014 that attempted to separate these two potential confounding factors.2 Instead of analyzing the error function over time, they cleverly investigated what happens on the error surface between a randomly initialized parameter vector and a successful final solution by using linear interpolation. So given a randomly initialized parameter vector θi and stochastic gradient descent (SGD) solution $$ {θ_f} $$ we aim to compute the error function at every point along the linear interpolation $$ {θ_a}=a.θ_f + (1-a).θ_i $$. In other words, they wanted to investigate whether local minima would hinder our gradient-based search method even if we knew which direction to move in. They showed that for a wide variety of practical networks with different types of neurons, the direct path between a randomly initialized point in the parameter space and a stochastic gradient descent solution isn’t plagued with troublesome local minima.

We can even demonstrate this ourselves using the feed-foward ReLU network. Using a checkpoint file that we saved while training our original feed-forward network, we can re-instantiate the inference and loss components while also maintaining a list of pointers to the variables in the original graph for future use in var_list_opt (where opt stands for the optimal   parameter settings):

In [1]:
import tensorflow as tf

In [12]:
def inference(x):  
    tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784, 10]) 
    b = tf.get_variable("b", [10])
    output = tf.nn.softmax(tf.matmul(x, W) + b) 
    return output 


def loss(output, y):
    dot_product = y * tf.log(output)
    # Reduction along axis 0 collapses each column into a  
    # single value, whereas reduction along axis 1 collapses 
    # each row into a single value. In general, reduction along  
    # axis i collapses the ith dimension of a tensor to size 1. 
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1) 
    loss = tf.reduce_mean(xentropy)
    return loss


def training(cost, global_step): 
    optimizer = tf.train.GradientDescentOptimizer(   
        learning_rate) 
    train_op = optimizer.minimize(cost,  
                                  global_step=global_step)  
    return train_op 

def evaluate(output, y):  
    correct_prediction = tf.equal(tf.argmax(output, 1),   
                                  tf.argmax(y, 1))   
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,       
                                      tf.float32)) 
    return accuracy


In [13]:
# mnist data image of shape 28*28=784

x=tf.placeholder("float",[None,784])
# 0-9 digits reognition => 10 classes
y=tf.placeholder("float",[None,10])

In [16]:
sess=tf.Session()

with tf.variable_scope("mlp_model",reuse=tf.AUTO_REUSE) as scope:
    output_opt=inference(x)
    cost_opt=loss(output_opt,y)
    saver=tf.train.Saver()
    scope.resuse_variables()
    
    var_list_opt=[
        "hidden_1/W",
        "hidden_1/b",
        "hidden_2/W",
        "hidden_2/b",
        "output/W",
        "output/b"
    ]
    
    var_list_opt=[tf.get_variable(v) for v in var_list_opt]
    saver.restore(sess,"mlp_logs/model-checkpoint-file")

AttributeError: 'VariableScope' object has no attribute 'resuse_variables'